# Solution from Tensorflow website
#### adapted from: https://www.tensorflow.org/get_started/estimator

## 1. Use Tensorflow to create model
Use Tensorflow to create a model to predict the species of Iris from a flower's sepal width, sepal length, petal width, and petal length.


## 2. Split the data into training and testing
Split the data set into a training set and a testing set.
You should investigate the best way to do this, and list any online references used in your notebook.
If you wish to, you can write some code to randomly separate the data on the fly.

In [2]:
# adapted from: https://www.tensorflow.org/get_started/estimator
# import all the necessary modules 
# and define where to download and store the dataset:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from six.moves.urllib.request import urlopen

import numpy as np
import tensorflow as tf

# Data sets
# define to download and store the dataset from tensorflow:
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

## 3. Train the model
Use the testing set to train your model

## 4. Test the model
Use the testing set to test your model, clearly calculating and displaying the error rate.

In [3]:
# adapted from: https://www.tensorflow.org/get_started/estimator
# Give some time to the code to run 
def main():
  # If the training and test sets aren't stored locally, download them.
  if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "wb") as f:
      f.write(raw)

  if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "wb") as f:
      f.write(raw)

#load the training and test sets into Datasets using the load_csv_with_header() 
#method in learn.datasets.base. The load_csv_with_header() method takes three required arguments:
#filename, which takes the filepath to the CSV file
#target_dtype, which takes the numpy datatype of the dataset's target value.
#features_dtype, which takes the numpy datatype of the dataset's feature values.  
#Load datasets.
    
  training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TRAINING,
      
#Here, the target (the value you're training the model to predict) is flower species,
#which is an integer from 0–2, so the appropriate numpy datatype is np.int:
      
      target_dtype=np.int,
      features_dtype=np.float32)
  test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TEST,
      target_dtype=np.int,
      features_dtype=np.float32)

  # Specify that all features have real-value data
  feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]

  # Build 3 layer DNN with 10, 20, 10 units respectively.
  classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                          hidden_units=[10, 20, 10],
                                          n_classes=3,
                                          model_dir="/tmp/iris_model")
  # Define the training inputs
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(training_set.data)},
      y=np.array(training_set.target),
      num_epochs=None,
      shuffle=True)

  # Train model.
  classifier.train(input_fn=train_input_fn, steps=2000)

#The tf.estimator API uses input functions,
#which create the TensorFlow operations that generate data for the model.
#We can use tf.estimator.inputs.numpy_input_fn to produce the input pipeline:    
#Define the test inputs

  test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(test_set.data)},
      y=np.array(test_set.target),
      num_epochs=1,
      shuffle=False)

  # Evaluate accuracy.
  accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

  print("\nTest Accuracy: {0:f}\n".format(accuracy_score))
  #Evaluate and print error rate. 
  print("\nError Rate: {0:f}\n".format(1 - accuracy_score))

  # Classify two new flower samples.
  new_samples = np.array(
      [[6.4, 3.2, 4.5, 1.5],
       [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
  predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": new_samples},
      num_epochs=1,
      shuffle=False)

  predictions = list(classifier.predict(input_fn=predict_input_fn))
  predicted_classes = [p["classes"] for p in predictions]

  print(
      "New Samples, Class Predictions:    {}\n"
      .format(predicted_classes))

if __name__ == "__main__":
    main()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-20000
INFO:tensorflow:Saving checkpoints for 20001 into /tmp/iris_model\model.ckpt.
INFO:tensorflow:loss = 1.95149, step = 20001
INFO:tensorflow:global_step/sec: 664.942
INFO:tensorflow:loss = 1.8436, step = 20101 (0.152 sec)
INFO:tensorflow:global_step/sec: 716.066
INFO:tensorflow:loss = 1.48299, step = 20201 (0.139 sec)
INFO:tensorflow:global_step/sec: 711.101
INFO:tensorflow:loss = 2.39913, step = 20301 (0.141 sec)
INFO:tensorflow:global_step/sec: 696.551
INFO:tensorflow:loss = 2.66413, step = 20401 (0.145 sec)
INFO:tensorflow:global_step/

## The second solution. 

The second solution completed using [Keras](https://keras.io/) libraries. This solution was originally adapted from: [salmanahmad4u](https://github.com/salmanahmad4u/keras-iris/blob/master/iris_nn.py)  and then step by step reworked during lab and lectures on [Emerging Technologies](https://emerging-technologies.github.io/) module with the lecturer [Ian McLoughlin](https://ianmcloughlin.github.io). Reworked code adapted from: https://github.com/emerging-technologies/keras-iris/blob/master/iris_nn.py

### Import neccessary imports to process iris date 

In [24]:
import numpy as np
import pandas as pd
import keras as kr
import csv 

### Using Panda import iris data from csv file

In [25]:
# Data from: https://github.com/mwaskom/seaborn-data/blob/master/iris.csv
#iris = pd.read_csv("data/iris.data.csv")
iris = list(csv.reader(open('IRIS.csv')))[1:]

### Create a list of values representing sepal length and width and petal length and width of various iris species. These will be our input values

In [26]:
# Goes through all the list and takes 4 values(0 to 3) in each array and sets them as np.float
inputs= np.array(iris)[:,:4].astype(np.float)#iris.values[:,:4].astype(np.float)

### Create a list of values representing the species of iris flower. These will be our output values ie. setosa versicolor and virginica

In [27]:
# Goes through all the list and takes last values(strings) in each array
outputs=np.array(iris)[:,4]

### Convert output strings to ints

In [28]:
# Convert the output strings to ints.
outputs_vals, outputs_ints = np.unique(outputs, return_inverse=True)

### Using keras utils we encode our catagory integers as binary catagorial variables.

In [29]:
# Encode the category integers as binary categorical vairables.
outputs_cats = kr.utils.to_categorical(outputs_ints)

### Split up the iris data randomly into training set data and test set data

In [30]:
# Split the input and output data sets into training and test subsets.
# create a random list if integers between 0 and 149
inds = np.random.permutation(len(inputs))
# Split the array into seperate arrays train_inds and test_inds
train_inds, test_inds = np.array_split(inds, 2)
# Using these inds create a training set of data and test set of data
inputs_train, outputs_train = inputs[train_inds], outputs_cats[train_inds]
inputs_test,  outputs_test  = inputs[test_inds],  outputs_cats[test_inds]

### Create a neural network

In [31]:
# Create a neural network.
model = kr.models.Sequential()

In [32]:
# Add an initial layer with 4 input nodes, and a hidden layer with 16 nodes.
model.add(kr.layers.Dense(16, input_shape=(4,)))

In [33]:
# Apply the sigmoid activation function to that layer.
model.add(kr.layers.Activation("sigmoid"))

In [34]:
# Add another layer, connected to the layer with 16 nodes, containing three output nodes.
model.add(kr.layers.Dense(3))

In [35]:
# Use the softmax activation function there.
model.add(kr.layers.Activation("softmax"))

In [36]:
# Configure the model for training.
# Uses the adam optimizer and categorical cross entropy as the loss function.
# Add in some extra metrics - accuracy being the only one.
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [37]:
# Fit the model using our training data.
model.fit(inputs_train, outputs_train, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
75/75 [==============================] - 0s 6ms/step - loss: 1.1399 - acc: 0.3867
Epoch 2/100
75/75 [==============================] - 0s 1ms/step - loss: 1.0867 - acc: 0.3867
Epoch 3/100
75/75 [==============================] - 0s 1ms/step - loss: 1.0387 - acc: 0.3867
Epoch 4/100
75/75 [==============================] - 0s 1ms/step - loss: 0.9936 - acc: 0.6800
Epoch 5/100
75/75 [==============================] - 0s 1ms/step - loss: 0.9407 - acc: 0.7333
Epoch 6/100
75/75 [==============================] - 0s 1ms/step - loss: 0.8846 - acc: 0.7333
Epoch 7/100
75/75 [==============================] - 0s 1ms/step - loss: 0.8318 - acc: 0.7333
Epoch 8/100
75/75 [==============================] - 0s 1ms/step - loss: 0.7809 - acc: 0.7333
Epoch 9/100
75/75 [==============================] - 0s 1ms/step - loss: 0.7394 - acc: 0.7333
Epoch 10/100
75/75 [==============================] - 0s 2ms/step - loss: 0.7005 - acc: 0.7333
Epoch 11/100
75/75 [==============================] - 0s 1m

75/75 [==============================] - 0s 1ms/step - loss: 0.1591 - acc: 0.9867
Epoch 88/100
75/75 [==============================] - 0s 1ms/step - loss: 0.1542 - acc: 0.9733
Epoch 89/100
75/75 [==============================] - 0s 1ms/step - loss: 0.1547 - acc: 0.9867
Epoch 90/100
75/75 [==============================] - 0s 1ms/step - loss: 0.1499 - acc: 1.0000
Epoch 91/100
75/75 [==============================] - 0s 1ms/step - loss: 0.1482 - acc: 0.9733
Epoch 92/100
75/75 [==============================] - 0s 1ms/step - loss: 0.1464 - acc: 0.9867
Epoch 93/100
75/75 [==============================] - 0s 1ms/step - loss: 0.1455 - acc: 0.9733
Epoch 94/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1432 - acc: 0.9867
Epoch 95/100
75/75 [==============================] - 0s 1ms/step - loss: 0.1395 - acc: 0.9733
Epoch 96/100
75/75 [==============================] - 0s 1ms/step - loss: 0.1410 - acc: 0.9867
Epoch 97/100
75/75 [==============================] - 0s 1ms/st

In [38]:
# Evaluate the model using the test data set.
loss, accuracy = model.evaluate(inputs_test, outputs_test, verbose=1)

74/74 [==============================] - 0s 475us/step


In [39]:
# Output the accuracy of the model.
print("\n\nLoss: %6.4f\tAccuracy: %6.4f" % (loss, accuracy))



Loss: 0.1622	Accuracy: 0.9595


In [40]:
# Predict the class of a single flower.
prediction = np.around(model.predict(np.expand_dims(inputs_test[0], axis=0))).astype(np.int)[0]
print("Actual: %s\tEstimated: %s" % (outputs_test[0].astype(np.int), prediction))
print("That means it's a %s" % outputs_vals[prediction.astype(np.bool)][0])

Actual: [1 0 0]	Estimated: [1 0 0]
That means it's a Iris-setosa


In [41]:
# Save the model to a file for later use.
model.save("iris_nn.h5")
# Load the model again with: model = load_model("iris_nn.h5")